<a href="https://colab.research.google.com/github/sriramk117/Open-GroundingDino/blob/main/tools/ADAFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting training data from FoodSeg103 to ODVG format

In [1]:
! pip install jsonlines

In [2]:
# Extract files from training zip folder
from zipfile import ZipFile

TRAINING_ZIP = '/content/train.zip'
ROOT_DIR = '/content/FoodSeg103'
TRAINING_PATH = '/content/FoodSeg103/train'

with ZipFile(TRAINING_ZIP, 'r') as zObject:
    zObject.extractall(
        path=ROOT_DIR)

In [3]:
import jsonlines
import os
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

TRAINING_SIZE = 103000
LABEL_LIST_PATH = '/content/category_id.txt'

# Create label dictionary
label_dict = {}
labels_file = open(LABEL_LIST_PATH)
lines = labels_file.read().splitlines()
labels_file.close()

for line in lines:
  words = line.split('\t')
  label = words[0]
  category = words[1]
  label_dict[label] = category

def get_label(c, im):
  # Find mean color of the contour and return corresponding label
  mask = np.zeros(im.shape[:2], np.uint8)
  cv.drawContours(mask, c, -1, 255, -1)
  mean = cv.mean(im, mask=mask)
  return round(mean[0])

def contour_to_bbox(c):
  # Get the bounding box of the contour
  pts = np.array(c)
  max_idx = np.argmax(pts, axis=0)
  min_idx = np.argmin(pts, axis=0)
  max_x_coord, max_y_coord = pts[max_idx][0]
  min_x_coord, min_y_coord = pts[min_idx][0]
  min_x = min_x_coord[0][0]
  min_y = min_y_coord[0][1]
  max_x = max_x_coord[0][0]
  max_y = max_y_coord[0][1]
  return [int(min_x), int(min_y), int(max_x), int(max_y)]

metas = []
for filename in os.listdir(TRAINING_PATH):
  f = os.path.join(TRAINING_PATH, filename)

  # Convert image to OpenCV image
  im = cv.imread(f)
  gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

  # Get contours
  contours , hierarchy = cv.findContours(np.array(gray, dtype=np.int32), cv.RETR_FLOODFILL, cv.CHAIN_APPROX_SIMPLE)
  print(len(contours))

  # Write detections data for this image
  instance_list = []
  for c in contours:
    bbox_xyxy = contour_to_bbox(c)
    label = get_label(c, gray)
    category = label_dict[str(label)]
    instance_list.append({
        "bbox": bbox_xyxy,
        "label": label,
        "category": category
    })

  h, w, _ = im.shape
  metas.append(
      {
          "filename": filename,
          "height": h,
          "width": w,
          "detection": {
              "instances": instance_list
          }
      }
  )

with jsonlines.open('train.jsonl', mode='w') as writer:
  writer.write_all(metas)



13
9
6
5
7
7
15
10
10
6
10
4
3
11
5
7
8
11
13
11
18
5
10
20
5
6
15
8
9
8
5
14
4
20
14
12
7
5
18
4
11
10
9
4
13
14
5
7
5
9
25
15
22
22
17
4
5
4
10
6
3
9
6
8
5
5
6
5
10
7
13
13
10
5
13
10
21
19
20
12
15
4
3
6
10
27
9
7
14
11
5
6
6
9
6
11
8
15
6
9
9
13
9
6
6
6
5
7
3
10
7
9
8
7
6
6
7
8
9
16
29
10
4
13
12
9
6
5
13
9
5
6
5
7
5
12
22
13
4
6
9
9
4
18
5
4
5
10
5
5
7
5
5
17
12
9
17
10
10
20
17
19
17
15
13
4
7
4
14
7
5
7
6
4
8
8
18
13
8
8
18
9
7
6
4
9
13
5
6
21
6
10
5
6
11
3
6
14
4
7
5
7
7
10
11
10
19
4
7
15
9
6
4
6
8
11
7
4
13
5
4
3
6
5
5
4
7
13
21
12
5
8
14
13
8
6
5
3
10
6
25
10
6
4
12
4
6
5
12
12
9
6
6
8
5
10
17
5
11
12
5
13
14
5
8
11
4
7
15
5
7
4
6
15
19
11
20
16
10
11
5
6
6
4
20
6
23
5
10
3
11
24
17
8
5
6
5
7
3
9
21
19
6
3
11
8
4
5
7
7
12
10
6
9
6
7
5
9
10
11
11
9
4
14
8
5
9
18
9
8
7
15
3
8
9
8
8
27
4
12
7
9
14
5
8
6
8
10
12
11
7
6
4
10
9
4
7
6
7
12
10
5
5
19
7
9
7
14
13
10
3
8
6
8
10
8
15
8
4
6
13
5
3
11
15
9
13
8
7
5
9
12
16
27
17
5
10
8
7
14
9
16
11
12
21
10
15
8
12
6
4
9
8
7
9
11
12
21
7